In [1]:
#imports used:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from collections import Counter

#not yet used, but imported:
from urllib.request import urlopen
import mechanicalsoup
import time
from requests.auth import HTTPBasicAuth


In [6]:
#my specific credentials for login
with open('payload.py', 'r') as file:
    exec(file.read())

In [ ]:
#create a specific session
with requests.session() as s: 
    
    #use GET command to find the authenticity token within the login url
    req = s.get(login_url).text 
    html = BeautifulSoup(req,"html.parser") 
    token = html.find("input", {"name": "authenticity_token"}).attrs["value"] 

    #use my specific authenticity token plus credentials to access my account
    payload = { 
        "authenticity_token": token, 
        'athlete[email]': login, 
        'athlete[password]': password, 
    }
    
    #response following posting my data to the location above
    res = s.post(login_url, data=payload) 

#report outcome of login activity
if res.ok or res.status_code == 302:
    redirected_url = res.url
    print("Login successful", res.status_code, redirected_url)
else:
    print("Login failed")

In [ ]:
#retrieve the contents of the dashboard site
r = s.get(dashboard_url) 
soup = BeautifulSoup (r.content, "html.parser")
print(dashboard_url, soup)

* All code above is used to locate, open, and read the wandrer.earth data. This must be executed properly before moving forward.
* `soup` is now a variable containing the html content of the site `dashboard_url`

In [ ]:
#make lists to add the region and progress data
region_gb_list = []
region_name_list = []
region_gps_list = []

region_completed_list = []
region_progress_list = []

#Find all elements with the class "achievement_row"
#In addition to locations, this will also check date achievements, but I can drop those in the df later
achievement_rows = soup.find_all(class_="achievement_row")

#set counters to check NaNs
no_name = 0
no_gb = 0
no_gps = 0

#iterate through each achievement row
for achievement_row in achievement_rows:
    #extract location text
    #if section has name, use that, then check missing name, otherwise 'N/A'
    
    present_name_element = achievement_row.select_one('.achievement_name')
    missing_name_element = achievement_row.select_one('.missing_achievement_name')

    if present_name_element:
        achievement_name = present_name_element.text
        
    elif missing_name_element:
        achievement_name = missing_name_element.text
    else:
        achievement_name = 'N/A'
        no_name = no_name + 1
    
    region_name_list.append(achievement_name)
        
    # Extract "gb ID" value
    data_gb_element = achievement_row.select_one('.geom_toggle')
    data_gb_value = data_gb_element['data-gb'] if data_gb_element else "N/A"
    if data_gb_value == "N/A":
        no_gb = no_gb + 1
    region_gb_list.append(data_gb_value)

    # Extract coordinates from the data-diagonal attribute
    data_diagonal_element = achievement_row.select_one('.geom_toggle')
    coordinates = eval(data_diagonal_element['data-diagonal'])['coordinates'] if data_diagonal_element else "N/A"
    if coordinates == "N/A":
        no_gps = no_gps + 1
    region_gps_list.append(coordinates)

    #print("Location:", achievement_name)
    #print("Data-gb Value:", data_gb_value)
    #print("Coordinates:", coordinates)
    #print("------------------------")

#show the number of missing values vs. found values
print(f"GB IDs \n missing: {no_gb}, present: {len(region_gb_list)-no_gb}")
print(f"Names: \n missing: {no_name}, present: {len(region_name_list)-no_name}")
print(f"Coordinates \n missing: {no_gps}, present: {len(region_gps_list)-no_gps}")

There are 172 valid Name calls, but only 162 valid Regions. The name calls are likely non-location achievements, which will be dropped in the subsequent df.

Next, I'll scrape sub-sites for each GB value to find the total distance available by plugging the GB value into a url and retrieving the result.

In [ ]:
for i in region_gb_list:
    gb_url = ('https://wandrer.earth/geometry_badges/'+i)
    r = s.get(gb_url) 
    sub_soup = BeautifulSoup(r.content, "html.parser")
    total_span = sub_soup.find('span', string=lambda s: 'km total length' in s)
    text = lambda f: f if f else 'NA'

    print(text(total_span))

In [ ]:
#prepare lists for data on distances:
#(by including the code in this block, it will prevent overloading redundant data in these lists if the code is re-run)
region_foot_distance_list = []
region_bike_distance_list = []
region_total_distance_list = []

#look for the summary pages of each of the regions represented by the gb codes, by scraping their individual URLs.
for i in region_gb_list:
    gb_url = ('https://wandrer.earth/geometry_badges/'+i)
    r = s.get(gb_url) 
    sub_soup = BeautifulSoup(r.content, "html.parser")

#retrieve the total distance on bike, foot, and total.
#some gb areas do not have breakdown by bike and foot, assume the total is equal to the distance on bike or foot.

    #foot distance
    foot_span = sub_soup.find('span', string=lambda s: 'foot' in s)

    #bike distance
    bike_span = sub_soup.find('span', string=lambda s: 'bike' in s)
    
    #total distance
        #this is an interesting case, because when bike and run distances are not presented, the string is different.
        #use an if statement to look for cases with all 3 listed, and else find the single total value.
    if sub_soup.find('span', string=lambda s: 'total length' in s):
        total_span = sub_soup.find('span', string=lambda s: 'total length' in s)
    else: total_span = sub_soup.find('span', string=lambda s: 'km. Worth' in s)

    #convert foot, bike, or total found in html to TEXT, strip excess characters
    text = lambda f: f.get_text().strip().replace(',', '') if f else 'NA'
    
    #Append the distances to new lists
    region_foot_distance_list.append(text(foot_span))
    region_bike_distance_list.append(text(bike_span))
    region_total_distance_list.append(text(total_span))

    print(f'{gb_url} \n    {text(foot_span)},\n    {text(bike_span)},\n    {text(total_span)}')

In [ ]:
for x in [region_gb_list, region_name_list, region_gps_list, region_foot_distance_list, region_bike_distance_list, region_total_distance_list]:
    print(len(x))

In [ ]:
data = {'GB code': region_gb_list, 'Locations': region_name_list, 'Coordinates': region_gps_list}
df = pd.DataFrame(data)
df.sample(20)

In [ ]:
#drop the time achievement rows
df.drop(df[df['GB code'] == 'N/A'].index, inplace=True)
df.shape

In [ ]:
df['Foot'] = region_foot_distance_list
df['Bike'] = region_bike_distance_list
df['Total distance'] = region_total_distance_list
df.shape, df.dtypes

In [ ]:
df.head()

The dataframe has been successfully created, the desired columns are present, and the information appears correct based on manual inspection of my account info online.

The data is all object type. The following updates are needed:

Column | Change needed
-|-
<b>GB code</b> | change to `int` dtype
<b>Locations</b> | none
<b>Coordinates</b> | 1. change to series dtype, 2 calculate midpoint, 3. change to `float` dtype
<b>Foot</b></b> | split to new columns: distance, points. Trim text and change to `float` dtype
<b>Bike</b> | split to new columns: distance, points. Trim text and change to `float` dtype
<b>Total distance</b> |split to new columns: distance, points. Trim text and change to `float` dtype


In [ ]:
#update the GB code to float
df['GB code'] = df['GB code'].astype('float64')
df.dtypes

In [ ]:
df.shape

In [ ]:

dfcoords = pd.DataFrame(df['Coordinates'].tolist(), index=df.index)
dfcoords

In [ ]:
import json
# Define a function to convert the string to a list of coordinates
def parse_coordinates(coord_str):
    coords = json.loads(coord_str)  # Using eval to convert the string to a list
    return pd.Series(coords)

# Apply the function to the 'Coordinates' column and expand into separate columns
df[['Longitude1', 'Latitude1', 'Longitude2', 'Latitude2']] = df['Coordinates'].apply(parse_coordinates)

# Drop the original 'Coordinates' column if needed
#df = df.drop('Coordinates', axis=1)

# Display the resulting DataFrame
print(df)

In [ ]:
inprogress_df = df.copy()
inprogress_df['Progress'] = inprogress_df['Progress'].astype(float)
inprogress_df['Region'] = inprogress_df['Region'].astype(str)
inprogress_df = inprogress_df[~inprogress_df['Region'].str.contains('2023')]
inprogress_df.sort_values(by='Progress', inplace=True)
inprogress_df.shape

Next, 
1. find the total distance of the given `Region`
2. IF 
 * distance >75%, find <u>the distance required to complete (>99%) region</u>
 * distance >50%, find <u>the distance required to move to >75%</u>
 * distance >25%, find <u>the distance required to move to >50%</u>
 * distance <25%, find <u>the distance required to move to >25%</u>
3. find the distance of regions that haven't been started required to move to 25%
4. sort by shortest distance to completion for the region

import re

for region in regions:
    
    # Extract text content from the div element
    region_text = region.get_text(separator=' ', strip=True)

    # Use regular expression to find the distance on foot
    match = re.search(r'(\d+\.\d+)\s*km on foot', region_text)

    if match:
        distance_value = match.group(1)
        print(distance_value)



In [ ]:
#identify the region distance element from the popup info on each region
region_code_list = []
region_code = r.find('a', {'class': 'fa fa-info-circle info-circle', 'href': '#', 'data-bs-toggle': 'modal'})

#look for progress
if region_progress_element:
    progress_bar = region_progress_element.find('div', class_='progress-bar')
    region_progress = progress_bar['aria-valuenow']
    region_progress_values.append(region_progress)
    #print(f"Region: {region_name}     Progress: {region_progress}% \n")
else: region_progress_values.append(0)


In [ ]:
#find number code for regions and return ID
if region_distance_element:
    region_distance = region_distance_element.text.strip()

    #add region name to list
    region_distances.append(region_distance)
else: print('none found')

In [ ]:
#make list to add the total region distance data
region_distances = []

for r in regions:
    distance_span = r.find('span', string=lambda s: 'km on foot' in s)
    
    if distance_span:
        # Extract the distance value
        distance_text = distance_span.get_text(strip=True)
        distance_value = distance_text.split()[0]  # Extract the numeric value
        print(distance_value)
    else:
        print("Distance on foot not found.")
    
    #find regions and return distance
    #if distance_element:
        #region_distance = distance_element.text.strip()
        
        #add region name to list
        #region_distances.append(region_distance)

In [ ]:
inprogress_df['Level_Up'] = 0.0

for index, row in inprogress_df.iterrows():
    progress = row['Progress']
    if progress >= 99.0:
        0
    elif progress >= 90.0:
        inprogress_df.at[index, 'Level_Up'] = 99.0 - progress
    elif progress >= 75.0:
        inprogress_df.at[index, 'Level_Up'] = 90.0 - progress
    elif progress >= 50.0:
        inprogress_df.at[index, 'Level_Up'] = 75.0 - progress
    elif progress >= 25.0:
        inprogress_df.at[index, 'Level_Up'] = 50.0 - progress
    else:
        inprogress_df.at[index, 'Level_Up'] = 25.0 - progress
        
inprogress_df.sort_values(by='Level_Up', inplace=True)
inprogress_df.head(12)

In [ ]:
import pandas as pd

# Assuming 'Progress' column exists in your DataFrame
# Create a DataFrame for demonstration purposes
dataa = {'Progress': [80, 60, 45, 30, 20]}
inprogres_df = pd.DataFrame(dataa)

# Apply the conditions and create the 'Level_Up' column
inprogres_df['Level_Up'] = 0  # Initialize the column with zeros

# Update the values based on conditions
inprogres_df.loc[inprogres_df['Progress'] >= 90, 'Level_Up'] = 100 - inprogres_df['Progress']
inprogres_df.loc[(inprogres_df['Progress'] >= 75) & (inprogres_df['Progress'] < 90), 'Level_Up'] = 90 - inprogress_df['Progress']
inprogres_df.loc[(inprogres_df['Progress'] >= 50) & (inprogres_df['Progress'] < 75), 'Level_Up'] = 75 - inprogress_df['Progress']
inprogres_df.loc[(inprogres_df['Progress'] >= 25) & (inprogres_df['Progress'] < 50), 'Level_Up'] = 50 - inprogress_df['Progress']
inprogres_df.loc[inprogres_df['Progress'] < 25, 'Level_Up'] = 25 - inprogres_df['Progress']

# Print the resulting DataFrame
print(inprogres_df)
